In [3]:
import numpy as np
import pandas as pd
import datetime
import math
from os import listdir
from os.path import isfile, join
import json

In [4]:
out='../output/'
inp='../input/monthly/'
pref='dam'

In [5]:
dat=pd.DataFrame(columns=[0,1,2])
onlyfiles = [f for f in listdir(inp) if isfile(join(inp, f))]
for f in onlyfiles:
    if (f.split('_')[0]==pref):
        #print(f)
        with open(join(inp, f), "r") as read_file:
            data = pd.DataFrame(json.load(read_file))
            dat=pd.concat([dat,data])

dat

,0,1,2
0,Date:Time,Price(USD/MWh),Turnover(MWh)
1,(2016/01/01) 00-01,40.1,477.5
2,(2016/01/01) 01-02,40.08,486.6
3,(2016/01/01) 02-03,40.06,495.6
4,(2016/01/01) 03-04,40.08,496.6
...,...,...,...
744,(2021/08/31) 23-24,45.09,266
745,Min,34.94,42.3
746,Max,303.23,375
747,Average,105.30967741935483870967741935,169.41787634408602150537634409


In [6]:
dat.drop(dat[dat[0] == 'Date:Time'].index, inplace = True)
dat.drop(dat[dat[0] == 'Min'].index, inplace = True)
dat.drop(dat[dat[0] == 'Max'].index, inplace = True)
dat.drop(dat[dat[0] == 'Average'].index, inplace = True)
dat.drop(dat[dat[0] == 'Total'].index, inplace = True)
dat.columns=['datehour','price','volume']
dat['price']=dat['price'].astype(float)
dat['volume']=dat['volume'].astype(float)
dat

,datehour,price,volume
1,(2016/01/01) 00-01,40.10,477.5
2,(2016/01/01) 01-02,40.08,486.6
3,(2016/01/01) 02-03,40.06,495.6
4,(2016/01/01) 03-04,40.08,496.6
5,(2016/01/01) 04-05,40.07,502.9
...,...,...,...
740,(2021/08/31) 19-20,254.20,185.0
741,(2021/08/31) 20-21,194.10,206.0
742,(2021/08/31) 21-22,99.83,213.0
743,(2021/08/31) 22-23,55.07,200.0


In [7]:
dat['date']=pd.to_datetime(dat.datehour.str.slice(1,11),format="%Y/%m/%d")
dat['hour']=dat.datehour.str.slice(13,15).astype(int)
dat

,datehour,price,volume,date,hour
1,(2016/01/01) 00-01,40.10,477.5,2016-01-01,0
2,(2016/01/01) 01-02,40.08,486.6,2016-01-01,1
3,(2016/01/01) 02-03,40.06,495.6,2016-01-01,2
4,(2016/01/01) 03-04,40.08,496.6,2016-01-01,3
5,(2016/01/01) 04-05,40.07,502.9,2016-01-01,4
...,...,...,...,...,...
740,(2021/08/31) 19-20,254.20,185.0,2021-08-31,19
741,(2021/08/31) 20-21,194.10,206.0,2021-08-31,20
742,(2021/08/31) 21-22,99.83,213.0,2021-08-31,21
743,(2021/08/31) 22-23,55.07,200.0,2021-08-31,22


In [8]:
dat=dat.drop(['datehour'],1)
dat

,price,volume,date,hour
1,40.10,477.5,2016-01-01,0
2,40.08,486.6,2016-01-01,1
3,40.06,495.6,2016-01-01,2
4,40.08,496.6,2016-01-01,3
5,40.07,502.9,2016-01-01,4
...,...,...,...,...
740,254.20,185.0,2021-08-31,19
741,194.10,206.0,2021-08-31,20
742,99.83,213.0,2021-08-31,21
743,55.07,200.0,2021-08-31,22


In [11]:
hourly=dat.set_index(pd.DatetimeIndex(dat['date']))
hourly['year']=hourly.index.year
hourly['month']=hourly.index.month
hourly['day']=hourly.index.day
hourly['day_of_week']=hourly.index.dayofweek
hourly=hourly.drop(['date'],axis=1)
hourly

,price,volume,hour,year,month,day,day_of_week
date,,,,,,,
2016-01-01,40.10,477.5,0,2016,1,1,4
2016-01-01,40.08,486.6,1,2016,1,1,4
2016-01-01,40.06,495.6,2,2016,1,1,4
2016-01-01,40.08,496.6,3,2016,1,1,4
2016-01-01,40.07,502.9,4,2016,1,1,4
...,...,...,...,...,...,...,...
2021-08-31,254.20,185.0,19,2021,8,31,1
2021-08-31,194.10,206.0,20,2021,8,31,1
2021-08-31,99.83,213.0,21,2021,8,31,1


In [13]:
hourly['value']=hourly['price']*hourly['volume']
daily=hourly.groupby(['year','month','day']).sum().drop(['hour','day_of_week'],axis=1)
daily['price_weight_av']=daily['value']/daily['volume']
daily=daily.drop(['price'], axis=1)
daily['price_max']=hourly[['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_min']=hourly[['year','month','day','price']].groupby(['year','month','day']).min()
daily['price_mean']=hourly[['year','month','day','price']].groupby(['year','month','day']).mean()

daily['power_max']=hourly[['year','month','day','volume']].groupby(['year','month','day']).max()
daily['power_min']=hourly[['year','month','day','volume']].groupby(['year','month','day']).min()
daily['power_mean']=hourly[['year','month','day','volume']].groupby(['year','month','day']).mean()

daily['price_morning_max']=hourly.loc[hourly['hour']<11][['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_evening_max']=hourly.loc[hourly['hour']>=11][['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_standard']=hourly.loc[(hourly['hour']>=12) & (hourly['hour']<=15)][['year','month','day','price']].groupby(['year','month','day']).mean()

daily

volume       value  price_weight_av  price_max  price_min  \
year month day                                                              
2016 1     1    7873.8  336882.508        42.785251      60.08      35.09   
           2    7319.5  345982.021        47.268532     118.49      40.03   
           3    7198.7  351712.733        48.857812     118.47      44.06   
           4    4457.0  225938.810        50.693024     177.66      46.70   
           5    3750.4  179068.882        47.746609     181.40      42.06   
...                ...         ...              ...        ...        ...   
2021 8     27   4642.3  470184.091       101.282574     250.05      43.09   
           28   4243.0  208117.500        49.049611     100.03      36.09   
           29   4831.0  217160.630        44.951486      76.82      36.08   
           30   3552.0  433410.420       122.018699     251.40      43.10   
           31   3957.0  531017.862       134.197084     254.20      45.09   

                price_mean  power_max  power_min  power_mean  \
year month day                                                 
2016 1     1     45.982500      502.9       53.0  328.075000   
           2     54.114167      486.0       40.0  304.979167   
           3     54.959167      486.0       40.0  299.945833   
           4     80.164583      486.0        0.0  185.708333   
           5     84.503333      436.0        0.0  156.266667   
...                    ...        ...        ...         ...   
2021 8     27   116.290417      328.0       70.0  193.429167   
           28    53.083750      260.0       91.0  176.791667   
           29    46.377000      315.0      128.0  241.550000   
           30   128.031000      290.0      133.0  177.600000   
           31   130.172000      295.1      126.0  197.850000   

                price_morning_max  price_evening_max  price_standard  
year month day                                                        
2016 1     1                48.03              60.08         45.8275  
           2                56.05             118.49         50.2275  
           3                56.01             118.47         50.6050  
           4                80.02             177.66         80.0200  
           5               148.40             181.40         80.0500  
...                           ...                ...             ...  
2021 8     27              203.00             250.05         97.3225  
           28              100.03             100.01         49.9550  
           29               50.04              76.82         47.2925  
           30              200.08             251.40         95.0700  
           31              199.07             254.20         95.8450  

[2070 rows x 12 columns]

In [14]:
hourly['cat']='off'

if (pref=='dam'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==6),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=7) & (hourly['hour']<=9),'cat']='morning'

    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=10) & (hourly['hour']<=16),'cat']='standard'

    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==17) & (hourly['month']>=6) & (hourly['month']<=8),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==17) & ((hourly['month']<=5) | (hourly['month']>=9)),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==18) & (hourly['month']<=3),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==18) & (hourly['month']>=4),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==19),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==20) & (hourly['month']<=3),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==20) & (hourly['month']>=4),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==21),'cat']='standard'

    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==17) & (hourly['month']>=6) & (hourly['month']<=8),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==18) & (hourly['month']>=4),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==19),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==20) & (hourly['month']<=3),'cat']='standard'

if (pref=='fpm'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=6) & (hourly['hour']<=21),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='standard'

if (pref=='fpw'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==5),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=6) & (hourly['hour']<=8),'cat']='morning'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=9) & (hourly['hour']<=17),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='evening'
    
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='standard'
    
hourly

,price,volume,hour,year,month,day,day_of_week,value,cat
date,,,,,,,,,
2016-01-01,40.10,477.5,0,2016,1,1,4,19147.750,off
2016-01-01,40.08,486.6,1,2016,1,1,4,19502.928,off
2016-01-01,40.06,495.6,2,2016,1,1,4,19853.736,off
2016-01-01,40.08,496.6,3,2016,1,1,4,19903.728,off
2016-01-01,40.07,502.9,4,2016,1,1,4,20151.203,off
...,...,...,...,...,...,...,...,...,...
2021-08-31,254.20,185.0,19,2021,8,31,1,47027.000,evening
2021-08-31,194.10,206.0,20,2021,8,31,1,39984.600,standard
2021-08-31,99.83,213.0,21,2021,8,31,1,21263.790,standard


In [15]:
daily['off_price_mean']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['standard_price_mean']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['morning_price_mean']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['evening_price_mean']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()

daily['off_hour_count']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['standard_hour_count']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['morning_hour_count']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['evening_hour_count']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()

daily['off_volume']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['standard_volume']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['morning_volume']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['evening_volume']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()

daily['off_value']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['standard_value']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['morning_value']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['evening_value']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()

daily

volume       value  price_weight_av  price_max  price_min  \
year month day                                                              
2016 1     1    7873.8  336882.508        42.785251      60.08      35.09   
           2    7319.5  345982.021        47.268532     118.49      40.03   
           3    7198.7  351712.733        48.857812     118.47      44.06   
           4    4457.0  225938.810        50.693024     177.66      46.70   
           5    3750.4  179068.882        47.746609     181.40      42.06   
...                ...         ...              ...        ...        ...   
2021 8     27   4642.3  470184.091       101.282574     250.05      43.09   
           28   4243.0  208117.500        49.049611     100.03      36.09   
           29   4831.0  217160.630        44.951486      76.82      36.08   
           30   3552.0  433410.420       122.018699     251.40      43.10   
           31   3957.0  531017.862       134.197084     254.20      45.09   

                price_mean  power_max  power_min  power_mean  \
year month day                                                 
2016 1     1     45.982500      502.9       53.0  328.075000   
           2     54.114167      486.0       40.0  304.979167   
           3     54.959167      486.0       40.0  299.945833   
           4     80.164583      486.0        0.0  185.708333   
           5     84.503333      436.0        0.0  156.266667   
...                    ...        ...        ...         ...   
2021 8     27   116.290417      328.0       70.0  193.429167   
           28    53.083750      260.0       91.0  176.791667   
           29    46.377000      315.0      128.0  241.550000   
           30   128.031000      290.0      133.0  177.600000   
           31   130.172000      295.1      126.0  197.850000   

                price_morning_max  ...  morning_hour_count  \
year month day                     ...                       
2016 1     1                48.03  ...                 3.0   
           2                56.05  ...                 NaN   
           3                56.01  ...                 NaN   
           4                80.02  ...                 3.0   
           5               148.40  ...                 3.0   
...                           ...  ...                 ...   
2021 8     27              203.00  ...                 3.0   
           28              100.03  ...                 NaN   
           29               50.04  ...                 NaN   
           30              200.08  ...                 3.0   
           31              199.07  ...                 3.0   

                evening_hour_count  off_volume  standard_volume  \
year month day                                                    
2016 1     1                   2.0      3851.5           2920.3   
           2                   NaN      5969.5           1350.0   
           3                   NaN      7198.7              NaN   
           4                   2.0      3828.0            464.0   
           5                   2.0      3230.0            368.4   
...                            ...         ...              ...   
2021 8     27                  3.0      2343.9           1338.4   
           28                  NaN      3242.0           1001.0   
           29                  NaN      4831.0              NaN   
           30                  3.0      1069.0           1476.0   
           31                  3.0       940.0           1774.8   

                morning_volume  evening_volume   off_value  standard_value  \
year month day                                                               
2016 1     1             993.0           109.0  152306.418      131289.730   
           2               NaN             NaN  271567.021       74415.000   
           3               NaN             NaN  351712.733             NaN   
           4             109.0            56.0  179880.680       32852.590   
           5             109.0    

In [19]:
daily.to_csv(out + pref + '_daily.csv')

In [20]:
hourly.to_csv(out + pref + '_hourly.csv')

In [21]:
hourly_flat=hourly.reset_index().set_index(['date'])
hourly_slim=hourly_flat.loc[(hourly_flat['year']<2021)][['hour','price','volume','cat']]
hourly_slim

,hour,price,volume,cat
date,,,,
2016-01-01,0,40.10,477.5,off
2016-01-01,1,40.08,486.6,off
2016-01-01,2,40.06,495.6,off
2016-01-01,3,40.08,496.6,off
2016-01-01,4,40.07,502.9,off
...,...,...,...,...
2020-12-31,19,160.03,60.0,evening
2020-12-31,20,60.06,40.0,standard
2020-12-31,21,60.08,64.0,standard


In [22]:
hourly_slim.to_csv(out + pref + '_hourly_slim.csv')

In [23]:
monthly=daily.groupby(['year','month']).mean().reset_index(['month','year'])
monthly=monthly.drop(['morning_volume','off_volume','standard_volume','evening_volume','price_weight_av'],1)
monthly.rename(columns={'value':'value_mean','volume':'volume_mean','off_hour_count':'off_hour_mean','standard_hour_count':'standard_hour_mean','morning_hour_count':'morning_hour_mean','evening_hour_count':'evening_hour_mean'}, inplace=True)
monthly[['price_max','standard_price_max','off_price_max','morning_price_max','evening_price_max']]=daily.groupby(['year','month']).max().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]
monthly[['price_min','standard_price_min','off_price_min','morning_price_min','evening_price_min']]=daily.groupby(['year','month']).min().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]
monthly[['price_stdev','standard_price_stdev','off_price_stdev','morning_price_stdev','evening_price_stdev']]=daily.groupby(['year','month']).std().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]

monthly[['volume','morning_volume','evening_volume','standard_volume','off_volume']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['volume','morning_volume','evening_volume','standard_volume','off_volume']]
monthly[['value','morning_value','evening_value','standard_value','off_value']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['value','morning_value','evening_value','standard_value','off_value']]

monthly[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]

monthly['price_weight_mean']=monthly['value'] / monthly['volume']
monthly['off_price_weight_mean']=monthly['off_value'] / monthly['off_volume']
monthly['standard_price_weight_mean']=monthly['standard_value'] / monthly['standard_volume']
monthly['morning_price_weight_mean']=monthly['morning_value'] / monthly['morning_volume']
monthly['evening_price_weight_mean']=monthly['evening_value'] / monthly['evening_volume']


monthly['off_power']=monthly['off_volume'] / monthly['off_hour_count']
monthly['morning_power']=monthly['morning_volume'] / monthly['morning_hour_count']
monthly['standard_power']=monthly['standard_volume'] / monthly['standard_hour_count']
monthly['evening_power']=monthly['evening_volume'] / monthly['evening_hour_count']

monthly

,year,month,volume_mean,value_mean,price_max,price_min,price_mean,power_max,power_min,power_mean,...,evening_hour_count,price_weight_mean,off_price_weight_mean,standard_price_weight_mean,morning_price_weight_mean,evening_price_weight_mean,off_power,morning_power,standard_power,evening_power
0,2016,1,3785.351613,211930.289323,106.684000,45.982500,82.458788,362.651613,17.606452,159.959113,...,42.0,55.986949,50.402420,81.002858,77.850006,137.720998,273.188920,33.409524,56.583459,37.350000
1,2016,2,3871.755172,261082.316897,123.798500,75.023333,91.797506,367.334483,23.231034,161.993793,...,42.0,67.432548,56.605271,106.547628,129.026815,156.801247,274.654573,12.280952,75.799614,42.580952
2,2016,3,2904.254839,187713.287290,108.651500,60.115833,88.953989,285.867742,18.232258,122.263199,...,46.0,64.633890,55.295628,98.717668,116.099471,107.341031,212.547024,15.502899,56.748043,34.786957
3,2016,4,1921.116667,118965.331100,114.641667,42.853500,85.579278,183.970000,7.236667,80.344000,...,42.0,61.925094,57.116312,104.192200,104.410448,139.262082,152.162463,7.793651,19.055263,4.438095
4,2016,5,1597.551613,89924.522194,105.676500,28.659583,79.317876,165.183871,4.661290,68.442769,...,44.0,56.288962,49.684509,106.019186,95.368696,140.331232,124.132670,10.831818,18.262963,4.170455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,2021,4,2688.473333,161123.687933,110.314000,30.873750,80.865714,185.530000,40.316667,112.990667,...,44.0,59.931295,37.210893,66.760284,142.181426,252.474165,132.883133,74.968182,109.424815,46.463636
64,2021,5,2501.206452,168242.250258,113.129583,34.269167,84.021403,162.409677,48.145161,106.544812,...,42.0,67.264440,37.138276,90.791796,145.617597,285.751219,130.365097,81.076190,87.254511,51.383333
65,2021,6,2408.913333,258848.577167,145.522500,52.754167,121.474997,183.256667,33.226667,101.923778,...,66.0,107.454500,56.446636,131.411501,197.932285,289.100821,114.970732,124.809091,90.660317,52.624242
66,2021,7,3174.064516,328619.638677,148.904500,35.070000,113.573901,213.532258,59.832258,134.900081,...,66.0,103.532753,50.103428,128.585369,194.682037,289.336072,147.687059,143.656061,128.534615,80.031818


In [24]:
if (pref=='dam'):
    monthly['evening_days']=daily.loc[daily['evening_hour_count']>0].groupby(['year','month']).count().reset_index(['month','year'])['volume']
    monthly['morning_days']=daily.loc[daily['morning_hour_count']>0].groupby(['year','month']).count().reset_index(['month','year'])['volume']
    monthly['days']=daily.groupby(['year','month']).count().reset_index(['month','year'])['volume']

monthly

,year,month,volume_mean,value_mean,price_max,price_min,price_mean,power_max,power_min,power_mean,...,standard_price_weight_mean,morning_price_weight_mean,evening_price_weight_mean,off_power,morning_power,standard_power,evening_power,evening_days,morning_days,days
0,2016,1,3785.351613,211930.289323,106.684000,45.982500,82.458788,362.651613,17.606452,159.959113,...,81.002858,77.850006,137.720998,273.188920,33.409524,56.583459,37.350000,21,21,31
1,2016,2,3871.755172,261082.316897,123.798500,75.023333,91.797506,367.334483,23.231034,161.993793,...,106.547628,129.026815,156.801247,274.654573,12.280952,75.799614,42.580952,21,21,29
2,2016,3,2904.254839,187713.287290,108.651500,60.115833,88.953989,285.867742,18.232258,122.263199,...,98.717668,116.099471,107.341031,212.547024,15.502899,56.748043,34.786957,23,23,31
3,2016,4,1921.116667,118965.331100,114.641667,42.853500,85.579278,183.970000,7.236667,80.344000,...,104.192200,104.410448,139.262082,152.162463,7.793651,19.055263,4.438095,21,21,30
4,2016,5,1597.551613,89924.522194,105.676500,28.659583,79.317876,165.183871,4.661290,68.442769,...,106.019186,95.368696,140.331232,124.132670,10.831818,18.262963,4.170455,22,22,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,2021,4,2688.473333,161123.687933,110.314000,30.873750,80.865714,185.530000,40.316667,112.990667,...,66.760284,142.181426,252.474165,132.883133,74.968182,109.424815,46.463636,22,22,30
64,2021,5,2501.206452,168242.250258,113.129583,34.269167,84.021403,162.409677,48.145161,106.544812,...,90.791796,145.617597,285.751219,130.365097,81.076190,87.254511,51.383333,21,21,31
65,2021,6,2408.913333,258848.577167,145.522500,52.754167,121.474997,183.256667,33.226667,101.923778,...,131.411501,197.932285,289.100821,114.970732,124.809091,90.660317,52.624242,22,22,30
66,2021,7,3174.064516,328619.638677,148.904500,35.070000,113.573901,213.532258,59.832258,134.900081,...,128.585369,194.682037,289.336072,147.687059,143.656061,128.534615,80.031818,22,22,31


In [29]:
monthly=monthly.set_index(['year','month'])

year  month
2016  1        101.019750
      2        111.702665
      3        108.064517
      4        107.509266
      5        102.916384
                  ...    
2021  4         84.220435
      5        100.432298
      6        135.362413
      7        130.231083
      8        120.317712
Name: standard_price_mean, Length: 68, dtype: float64

In [32]:
monthly['off_price_diff']=monthly['off_price_mean']-monthly['off_price_mean'].shift(1)
monthly['morning_price_diff']=monthly['morning_price_mean']-monthly['morning_price_mean'].shift(1)
monthly['standard_price_diff']=monthly['standard_price_mean']-monthly['standard_price_mean'].shift(1)
monthly['evening_price_diff']=monthly['evening_price_mean']-monthly['evening_price_mean'].shift(1)

In [33]:
monthly.to_csv(out + pref + '_monthly.csv')

In [60]:
monthly_filt=monthly.loc[monthly['year']<2020]
daily_filt=daily.reset_index().loc[daily.reset_index()['year']<2020]

In [61]:
calmonthly=daily_filt.groupby(['month']).mean().reset_index()
calmonthly=calmonthly.drop(['year','day','volume','standard_volume','off_volume','morning_volume','evening_volume'],1)
calmonthly.rename(columns={'value':'value_mean','volume':'volume_mean','price_weight_av': 'price_weight_mean','off_hour_count':'off_hour_mean','standard_hour_count':'standard_hour_mean','morning_hour_count':'morning_hour_mean','evening_hour_count':'evening_hour_mean'}, inplace=True)

calmonthly[['price_median', 'off_price_median','standard_price_median', 'morning_price_median', 'evening_price_median']]=daily_filt.groupby(['month']).median().reset_index()[['price_mean','off_price_mean','standard_price_mean', 'morning_price_mean', 'evening_price_mean']]
calmonthly[['price_stdev', 'off_price_stdev','standard_price_stdev', 'morning_price_stdev', 'evening_price_stdev']]=daily_filt.groupby(['month']).std().reset_index()[['price_mean','off_price_mean','standard_price_mean', 'morning_price_mean', 'evening_price_mean']]

calmonthly[['volume', 'off_volume','standard_volume', 'morning_volume', 'evening_volume']]=monthly_filt.groupby(['month']).mean().reset_index()[['volume','off_volume','standard_volume', 'morning_volume', 'evening_volume']]
calmonthly[['value', 'off_value','standard_value', 'morning_value', 'evening_value']]=monthly_filt.groupby(['month']).mean().reset_index()[['value','off_value','standard_value', 'morning_value', 'evening_value']]
calmonthly[['off_power','standard_power', 'morning_power', 'evening_power']]=monthly_filt.groupby(['month']).mean().reset_index()[['off_power','standard_power', 'morning_power', 'evening_power']]

calmonthly

,month,day_of_week,value_mean,price_weight_mean,price_max,price_min,price_mean,power_max,power_min,power_mean,...,volume,off_volume,standard_volume,morning_volume,evening_volume,value,off_power,standard_power,morning_power,evening_power
0,1,65.806452,3773.648435,40.405970,95.004839,27.899355,53.172812,5.358065,2.546774,3.714879,...,2656.750000,1677.400000,648.350000,190.200000,140.800000,1.169831e+05,4.948083,2.512984,2.756522,3.060870
1,2,70.612245,2417.153388,56.560000,102.117347,28.066122,66.069820,9.685714,0.000000,2.876888,...,1627.100000,1295.500000,321.350000,10.250000,0.000000,5.922026e+04,5.266260,1.879240,0.227778,0.000000
2,3,70.617284,8292.535185,27.453025,76.193457,26.825802,49.678801,21.814815,4.456790,11.275720,...,7232.000000,6198.666667,848.666667,117.333333,67.333333,2.238985e+05,23.299182,4.853041,2.607407,2.244444
3,4,67.062500,10068.978281,30.693082,68.357812,27.090313,46.758288,20.775000,4.718750,11.604297,...,5590.000000,3853.333333,1072.533333,372.733333,291.400000,2.148049e+05,28.017789,13.517402,19.153247,22.330519
4,5,68.578313,15639.570108,38.174440,74.521807,29.157349,49.104466,25.832530,8.522892,14.574056,...,9485.100000,5120.200000,2563.733333,994.166667,807.000000,4.326948e+05,18.893467,13.785081,20.281320,24.573430
5,6,70.046512,19799.947163,42.372336,113.796512,32.144884,64.339472,37.974419,7.417442,19.784041,...,12912.200000,10506.533333,2062.333333,238.333333,105.000000,5.675985e+05,31.126020,9.162252,3.968783,2.619841
6,7,67.182796,21391.251355,45.643103,125.730538,35.008387,71.012225,36.073118,7.515054,19.550448,...,14032.466667,11080.133333,2241.000000,425.666667,285.666667,6.631288e+05,31.936901,9.037157,6.605820,6.575397
7,8,69.247312,21718.561763,55.064815,131.696882,35.502688,71.143965,27.655914,7.647312,17.298790,...,12375.466667,7892.666667,3128.466667,799.000000,555.333333,6.732754e+05,23.132539,12.171248,11.639218,12.072464
8,9,70.577778,33599.931111,51.947519,92.956000,30.823556,54.781711,54.683333,12.066667,30.563565,...,21205.833333,15340.000000,4275.833333,951.666667,638.333333,1.007998e+06,44.033119,17.810721,15.535185,15.802778
9,10,66.064516,19082.346978,48.126553,83.135914,29.961613,53.965246,19.497849,10.026882,17.083799,...,12249.833333,6789.266667,4345.400000,758.500000,356.666667,5.915528e+05,19.596024,17.207435,11.325648,8.043478


In [62]:
calmonthly.to_csv(out + pref + '_calmonthly.csv')

In [63]:
calmonthly_hours=hourly.groupby(['month','hour']).mean()
calmonthly_hours=calmonthly_hours.drop(['year','day','day_of_week'],1)
calmonthly_hours

price     volume        value
month hour                                    
1     0      38.381087  11.568478   509.468380
      1      38.381087  11.568478   509.468380
      2      38.381087  11.568478   509.468380
      3      38.381087  11.568478   509.468380
      4      38.316935  11.626613   513.657411
...                ...        ...          ...
12    19    121.692500  14.888710   853.341532
      20     65.951855  25.118548  1396.648806
      21     65.951855  25.118548  1396.648806
      22     35.736452  46.086290  1868.078177
      23     35.736452  46.086290  1868.078177

[288 rows x 3 columns]

In [64]:
calmonthly_hours.to_csv(out + pref + '_calmonthly_hours.csv')